In [12]:
import pandas as pd
url = 'C:/Users/Haifa Elhorra/Desktop/ML/Projet_ML/DATAset/data_url.csv' # Définir le chemin d'accès au fichier CSV contenant les données.
url_csv = pd.read_csv(url, delimiter=',', on_bad_lines='skip')# Lire le fichier CSV en ignorant les lignes incorrectes ou mal formatées. Le délimiteur des colonnes est une virgule. 

In [13]:
url_df = pd.DataFrame(url_csv) # Convertir les données lues en un DataFrame pandas pour une manipulation plus facile.

In [14]:
import random
import numpy as np # Importer la bibliothèque numpy pour le calcul et random pour générer de l'aléatoire.
url_df = np.array(url_df)  # Convertir le DataFrame pandas en tableau numpy.
random.shuffle(url_df) # Mélanger les lignes du tableau de manière aléatoire.

In [15]:
y = [d[1] for d in url_df]
# Créer une liste contenant les valeurs de la cible colonne de chaque ligne.  
urls = [d[0] for d in url_df]
# Créer une liste contenant les valeurs des URLs colonne de chaque ligne. 

In [16]:
def sanitization(web): # Définir une fonction de nettoyage des données pour les URL.
    web = web.lower()  # Convertir l'URL en minuscules pour standardiser les caractères.
    token = [] # Initialiser une liste vide pour stocker les jetons (tokens) extraits.
    dot_token_slash = [] # Initialiser une liste vide pour stocker les jetons avec des points et des barres obliques.
    raw_slash = str(web).split('/') # Diviser l'URL en segments séparés par des '/'
    for i in raw_slash: # Boucler à travers chaque segment de l'URL séparée par '/'
        raw1 = str(i).split('-')  # Diviser chaque segment par des '-' pour extraire les sous-parties.
        slash_token = [] # Initialiser une liste vide pour stocker les jetons de chaque segment.
        for j in range(0,len(raw1)): # Boucler à travers chaque sous-partie extraite '-'
            raw2 = str(raw1[j]).split('.') # Diviser chaque sous-partie par des '.' pour extraire des segments supplémentaires.
            slash_token = slash_token + raw2  # Ajouter les segments divisés par point à la liste `slash_token`.
        dot_token_slash = dot_token_slash + raw1 + slash_token  # Ajouter les segments séparés par tiret et les segments séparés par point à la liste `dot_token_slash`.
    token = list(set(dot_token_slash))  # Créer une liste de jetons uniques à partir des segments extraits.
    if 'com' in token:  # Vérifier si le jeton 'com' est présent dans la liste des jetons.
        token.remove('com') # Supprimer le jeton 'com' s'il est présent dans la liste.
    return token # Retourner la liste des jetons après nettoyage.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer # Importer le TfidfVectorizer pour transformer le texte en vecteurs numériques.
vectorizer = TfidfVectorizer(tokenizer=sanitization) # Créer une instance de TfidfVectorizer en utilisant la fonction de nettoyage `sanitization` comme tokenizer.

In [27]:
from sklearn.model_selection import train_test_split # Importer la fonction pour diviser les données en ensembles d'entraînement et de test.
x = vectorizer.fit_transform(urls) # Appliquer le TfidfVectorizer aux URL pour obtenir des vecteurs de caractéristiques.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) # Diviser les vecteurs de caractéristiques `x` et les étiquettes `y` en ensembles d'entraînement (80%) et de test (20%).

In [28]:
from sklearn.linear_model import LogisticRegression # Importer le modèle de régression logistique de la bibliothèque sklearn.
lgr = LogisticRegression(solver='lbfgs', max_iter=1000)   # Créer une instance du modèle de régression logistique avec le solveur 'lbfgs' et un nombre maximal d'itérations de 1000.
lgr.fit(x_train, y_train) # Entraîner le modèle de régression logistique avec les données d'entraînement (x_train et y_train).
score = lgr.score(x_test, y_test) # Évaluer le modèle sur les données de test (x_test et y_test) et obtenir le score (exactitude).
print("accuracy: {0:.4f}".format(score))  # Afficher l'exactitude du modèle avec une précision de 4 décimales.
vectorizer_save = vectorizer # Sauvegarder l'objet `vectorizer` (utilisé pour transformer le texte) dans une nouvelle variable `vectorizer_save`.

accuracy: 0.9827


In [29]:
import pickle # Importer le module pickle pour la sérialisation et la sauvegarde d'objets Python.
file = "pickel_model.pkl" # Définir le nom du fichier dans lequel le modèle sera sauvegardé.
with open(file, 'wb') as f: # Ouvrir le fichier en mode binaire pour l'écriture.
    pickle.dump(lgr, f) # Sauvegarder le modèle `lgr` dans le fichier.
f.close() # Fermer le fichier après avoir sauvegardé le modèle.

file2 = "pickel_vector.pkl" # Définir le nom du fichier pour sauvegarder le `vectorizer`.
with open(file2,'wb') as f2:     # Ouvrir le fichier en mode binaire pour l'écriture.
    pickle.dump(vectorizer_save, f2) # Sauvegarder l'objet `vectorizer_save` dans le fichier.
f2.close() # Fermer le fichier après avoir sauvegardé le `vectorizer`.